In [711]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

In [712]:
df = pd.read_csv('btcusd.csv')
#Todas as colunas para maiúsculas
df = df.rename(columns=str.lower)
#Tratamento das datas em segundos
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.dropna(subset=['datetime'])
df['Open'] = df['open']
df['High'] = df['high']
df['Low'] = df['low']
df['Close'] = df['close']
df['Volume'] = df['volume']

In [713]:
df_resample = df
df_resample['datetime'] = pd.to_datetime(df_resample['datetime'])
df_resample=df_resample.set_index('datetime')

df_fif_min = df_resample.resample('15min').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_hourly = df_resample.resample('h').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_six_hourly = df_resample.resample('6h').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_daily = df_resample.resample('D').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_monthly = df_resample.resample('ME').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

In [714]:
df_fif_min = df_fif_min.dropna(subset=['Open'])
df_hourly = df_hourly.dropna(subset=['Open'])
df_six_hourly = df_six_hourly.dropna(subset=['Open'])
df_daily = df_daily.dropna(subset=['Open'])
df_monthly = df_monthly.dropna(subset=['Open'])

In [802]:
def calculando_sinais(df):
    df['ema_50'] = ta.ema(df['Close'], length=50)
    stoch = ta.stoch(df['High'], df['Low'], df['Close'], window=14, smooth_k=3, smooth_d=1)
    df['stoch_k'] = stoch['STOCHk_14_3_3']#azul
    df['stoch_d'] = stoch['STOCHd_14_3_3']#laranja
    df['OBV'] = ta.obv(close=df['Close'], volume=df['Volume'])

    df["EMA50"] = ta.ema(df.Close, length=50)
    df["EMA100"] = ta.ema(df.Close, length=100)
    df["EMA150"] = ta.ema(df.Close, length=150)

    backrollingN = 10
    df['slopeEMA50'] = df['EMA50'].diff(periods=1)
    df['slopeEMA50'] = df['slopeEMA50'].rolling(window=backrollingN).mean()
    
    df['slopeEMA100'] = df['EMA100'].diff(periods=1)
    df['slopeEMA100'] = df['slopeEMA100'].rolling(window=backrollingN).mean()
    
    df['slopeEMA150'] = df['EMA150'].diff(periods=1)
    df['slopeEMA150'] = df['slopeEMA150'].rolling(window=backrollingN).mean()

    conditions = [
    ( (df['EMA50']<df['EMA100']) & (df['EMA100']<df['EMA150']) & (df['slopeEMA50']<0) & (df['slopeEMA100']<0) & (df['slopeEMA150']<0) ),
    ( (df['EMA50']>df['EMA100']) & (df['EMA100']>df['EMA150']) & (df['slopeEMA50']>0) & (df['slopeEMA100']>0) & (df['slopeEMA150']>0) )
           ]
    choices = [1, 2]
    df['EMAsignal'] = np.select(conditions, choices, default=0)

    TotSignal = [0] * len(df)
    for row in range(0, len(df)):
        TotSignal[row] = 0
        if df.EMAsignal[row]==1 and df.Open[row]>df.EMA50[row] and df.Close[row]<df.EMA50[row]:
            TotSignal[row]=-1
        if df.EMAsignal[row]==2 and df.Open[row]<df.EMA50[row] and df.Close[row]>df.EMA50[row]:
            TotSignal[row]=1
    
    df['signal']=TotSignal

    df['distancia_max'] = ((df['Close'] - df['ema_50']).abs()).rolling(window=4).max().fillna((df['Close'] - df['ema_50']).abs())
    df['distancia_min'] = ((df['Close'] - df['ema_50']).abs()).rolling(window=4).min().fillna((df['Close'] - df['ema_50']).abs())

    entrada = ((ta.increasing(df['stoch_k'])) 
    & (df['stoch_k'] >= 80)
    & (ta.cross(df['stoch_d'], df['stoch_k'], above=True))               
    & (ta.cross(df['Close'], df['ema_50'], above=True))   
    & (ta.increasing(df['OBV']))) 
  
    saida = ((ta.decreasing(df['stoch_k'])) 
    & (df['stoch_k'] <= 20)
    & (ta.cross(df['stoch_d'], df['stoch_k'], above=False))     
    & (ta.cross(df['Close'], df['ema_50'], above=False))
    & (ta.decreasing(df['OBV'])))

    df['target'] = 0  # Inicialmente definimos como "Esperar"
    df.loc[entrada, 'target'] = 1  # Sinal de Compra
    df.loc[saida, 'target'] = -1  # Sinal de Venda
    
    df['atr'] = ta.atr(df['High'], df['Low'], df['Close'], length=14) 
    
    df['stop'] = 0.0  # Inicia a coluna 'stop' com valores None
    df.loc[df['target'] != 0, 'stop'] = df['Close'].shift(3)# - df['atr'].shift(1)*1 

    df['take'] = 0.0  # Inicia a coluna 'stop' com valores None
    df.loc[df['target'] != 0, 'take'] =  df['Close'] + df['atr'].shift(1)*4 
    
    df = df.reset_index()
    return df[['datetime', 'Open' ,'High' ,'Low' ,'Close' ,'Volume','stoch_k','stoch_d','ema_50','OBV','distancia_max', 'distancia_min','stop','take', 'target','signal']]

In [803]:
df_tratado = calculando_sinais(df_six_hourly)
df_tratado = df_tratado[df_tratado['datetime'] >= datetime.datetime(2023,1,1)]

C:\Users\Mariano\AppData\Local\Temp\ipykernel_8944\465079413.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.EMAsignal[row]==1 and df.Open[row]>df.EMA50[row] and df.Close[row]<df.EMA50[row]:
C:\Users\Mariano\AppData\Local\Temp\ipykernel_8944\465079413.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.EMAsignal[row]==2 and df.Open[row]<df.EMA50[row] and df.Close[row]>df.EMA50[row]:


In [804]:
def SIGNAL():
    return df_tratado.target

In [805]:
def BSIGNAL():
    return df_tratado.signal

In [806]:
class SmaCross(Strategy):
    
    def init(self):
        self.sinal = self.I(SIGNAL)
        self.bsinal = self.I(BSIGNAL)

    def next(self):
        tp = self.data.take
        sl = self.data.stop
        if self.sinal == 1:
            self.buy(sl=sl)
        elif self.sinal == -1:
            self.sell(sl=sl)


bt = Backtest(df_tratado, SmaCross,
              cash=1000000, commission=.002,
              exclusive_orders=True)

output = bt.run()
#bt.plot()

C:\Users\Mariano\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
C:\Users\Mariano\AppData\Local\Temp\ipykernel_8944\3522738471.py:16: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_tratado, SmaCross,


In [807]:
output

Start                                 16071.0
End                                   18635.0
Duration                               2564.0
Exposure Time [%]                         0.0
Equity Final [$]                    1000000.0
Equity Peak [$]                     1000000.0
Return [%]                                0.0
Buy & Hold Return [%]               314.13958
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [808]:
bt.plot()

GridPlot(id='p18843', ...)